# FMQA によるモデル超電導材料の探索

FMQA の効果的な活用方法を理解していただくために、本サンプルコードでは、疑似的な材料から構成される超電導材料の探索を例題として取り扱います。

なお、本 FMQA サンプルコードでは、非線形なモデル代数式に基づいて、材料探索を行いますが、モデル代数式の代わりに、高精度なシミュレーションや実験計測結果を用いても同様のステップで様々な材料探索に関する FMQA 最適化を行うことが可能で、その場合、本サンプルコードをほぼそのまま活用いただけます。

ブラックボックス最適化や FMQA の基本知識については、『[量子アニーリング・イジングマシンによるブラックボックス最適化](./fmqa_0_algebra.ipynb)』をご覧ください。

また、FMQA を活用したより応用的なモデルケースとして、

- [FMQA による化学プラントにおける生産量最大化](./fmqa_2_reactor.ipynb)
- [FMQA と流体シミュレーションによる翼形状の最適化](./fmqa_3_aerofoil.ipynb)

も紹介されていますので、ご覧ください。

本ノートブックは、以下の構成となっています。

- 1\. [問題設定](#1)
  - 1.1\. [超電導材料の探索シナリオ](#1_1)
  - 1.2\. [乱数の初期化](#1_2)
  - 1.3\. [臨界温度モデルの定義](#1_3)
- 2\. [FMQA のプログラム実装](#2)
  - 2.1\. [クライアントの設定](#2_1)
  - 2.2\. [PyTorch による FM の実装](#2_2)
  - 2.3\. [初期教師データの作成](#2_3)
  - 2.4\. [FMQA サイクルの実行クラス](#2_4)
- 3\. [FMQA 実行例](#3)
  - 3.1\. [FMQA による最高臨界温度を実現する材料探索](#3_1)
  - 3.2\. [FMQA 最適化過程における目的関数値の推移](#3_2)
  - 3.3\. [本 FMQA サンプルコード実行例](#3_3)
- 発展：[より理解を深めるための練習問題](#4)
  - [発展1](#4_1)
  - [発展2](#4_2)
  - [発展3](#4_3)

<a id="1"></a>
## 1\. 問題設定

<a id="1_1"></a>
### 1.1\. 超電導材料の探索シナリオ

超電導技術は、リニアモーターカーに代表される輸送分野や計測分野、エネルギー分野においての活用が期待される技術で、現在様々な超電導を実現する超電導材料の開発が行われています。

しかし、現在確認されている超電導材料において、一般的に超電導状態に転移する温度（臨界温度）は絶対温度 0 K（ケルビン）付近であるため、超電導の活用には高コストな冷却が必要で、現状、社会的な応用は限られています。したがって、高温超電導体の探索が喫緊の課題です。

通常、超電導を実現する材料の探索には、数々の材料を選択・合成し、その合成材料の臨界温度を計測により評価するというプロセスを繰り返し、より高温の臨界温度を実現する合成対象の材料を同定する、という試行錯誤を行います。この合成と臨界温度の評価は非常に高時間コストと考えられます。この探索に対してブラックボックス最適化手法の1つである FMQA を活用し、比較的少ない評価回数で最適解に近い材料の組み合わせを求めます。

本サンプルコードでは、FMQA による材料探索の解説のために、疑似的な材料から構成される超電導材料の探索を例題として取り扱い、臨界温度の評価には模擬的な臨界温度モデルを用います。従って、以下で紹介する臨界温度モデル及び取得される材料の組み合わせは、必ずしも物理的な正確性を持たないことに注意してください。

<a id="1_2"></a>
### 1.2\. 乱数の初期化

実行毎にモデル出力や機械学習結果が変わらないようにするための、乱数seed値の初期化関数 `seed_everything()` を定義します。

In [ ]:
import os
import torch
import numpy as np


def seed_everything(seed=0):
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

<a id="1_3"></a>
### 1.3\. 臨界温度モデルの定義


本サンプルコードでは、$D$ 種類の材料から、いくつかの材料を組み合わせを上手く選択し、それらの合成で生成された超電導材料の臨界温度を最大化する最適化を実施します。

一般的に、臨界温度は実験計測で評価するしかなく、その実施には毎回比較的大きなコスト（時間・費用）が必要です。

本サンプルコードでは、臨界温度の計測の代わりに、以下の模擬的な臨界温度モデル `supercon_temperature()` を用いて評価を行いますが、この関数はあくまでも実験の代用であり、その中身やパラメータについては未知であるとして扱い、`supercon_temperature()` を呼ぶ回数にも制限があるものとして取り扱います。

In [ ]:
import numpy as np
import math

# 臨界温度計算のための種々の係数テーブルを乱数により決定する関数。


def set_properties(size):
    mu, sigma, ratio = 0.0, 1.0, 0.2
    table1 = np.random.rand(size) * 1e5 * (0.1 * math.log(size) - 0.23)
    table2 = np.random.lognormal(mu, sigma, size) * ratio
    table3 = np.random.lognormal(mu, sigma, size) * ratio
    return table1, table2, table3


# 与えられた材料の組み合わせ x (numpyの1次元配列) と各材料の物性値から、合成される超電導物質の臨界温度を計算するモデル関数。


def supercon_temperature(x, debye_table, state_table, interaction_table):
    debye_temperature = np.sum(x * debye_table) / np.sum(x)
    state_density = np.sum(x * state_table) / np.sum(x)
    interaction = np.sum(x * interaction_table) / np.sum(x)
    crit_temp = debye_temperature * math.exp(-1.0 / state_density / interaction)
    return crit_temp

以下では、上記で定義した臨界温度のモデル関数 `supercon_temperature(x)` を用いて、ランダムに選択した材料から合成される超電導材料の臨界温度を評価します。ここで、`D` は選択対象となる材料の数で、入力のバイナリベクトル `x` は、サイズ `D` のバイナリベクトルです。

例えば、5種類の材料から最初と最後の材料を選択して合成する、という場合、入力ベクトルは `x = [1, 0, 0, 0, 1]` となります。この場合、選択の仕方（組み合わせ）は、$2^5-1=31$ 通りあります。

`D = 100` の場合、組み合わせの数は、$10^{30}$ 通り程度存在し、全探索的な方法は困難と考えられます。

In [ ]:
seed_everything()  # 乱数シードの初期化
D = 100  # 決定変数のサイズ（材料選択肢の数）
debye_temperature_table, state_density_table, interaction_table = set_properties(
    D
)  # 係数テーブルの準備

# ランダムな入力 x で supercon_temp() 関数を n_cycle 回評価し、得られた最高臨界温度と平均臨界温度を出力。
n_cycle = 100
t_max = 0.0  # 臨界温度の最大値を格納する変数
t_mean = 0.0  # 臨界温度の平均値を計算する変数
for i in range(n_cycle):
    x = np.random.randint(0, 2, D)
    if np.sum(x) == 0:
        continue
    t_c = supercon_temperature(
        x, debye_temperature_table, state_density_table, interaction_table
    )
    if t_max < t_c:
        t_max = t_c
    t_mean += t_c
t_mean /= n_cycle

print(f"Max. critical temperature: {t_max:.2f} K")
print(f"Mean critical temperature: {t_mean:.2f} K")
print(f"{n_cycle=}")

<a id="2"></a>
## 2\. FMQA のプログラム実装

ここでは、FMQA のプログラム実装を行います。FMQA部分の実装は、『[量子アニーリング・イジングマシンによるブラックボックス最適化](./fmqa_0_algebra.ipynb)』と同一ですので、詳細はそちらの解説をご覧ください。

<a id="2_1"></a>
### 2.1\. クライアントの設定

Amplify のクライアントを作成し、必要なパラメータを設定します。 以下では、イジングマシンによる一度の探索時間を1秒に設定しています。

In [ ]:
from amplify.client import FixstarsClient

client = FixstarsClient()
client.parameters.timeout = 1000  # タイムアウト1秒
# client.token = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # ローカル環境等で使用する場合は、Amplify AEのアクセストークンを入力してください。

<a id="2_2"></a>
### 2.2\. PyTorch による FM の実装

FM の学習と推論を PyTorch で行います。`TorchFM` クラスでは、機械学習モデルとしての獲得関数 $g(\boldsymbol{x})$ を定義します。

In [ ]:
import torch.nn as nn


class TorchFM(nn.Module):
    def __init__(self, d: int, k: int):
        super().__init__()
        self.V = nn.Parameter(torch.randn(d, k), requires_grad=True)
        self.lin = nn.Linear(d, 1)  # 右辺第1項及び2項は全結合ネットワーク

    def forward(self, x):
        out_1 = torch.matmul(x, self.V).pow(2).sum(1, keepdim=True)
        out_2 = torch.matmul(x.pow(2), self.V.pow(2)).sum(1, keepdim=True)
        out_inter = 0.5 * (out_1 - out_2)
        out_lin = self.lin(x)
        out = out_inter + out_lin
        return out

次に、入出力データから FM を機械学習する関数 `train()` を定義します。一般的な機械学習と同様に、教師データを学習データと検証データに分割し、学習データを用いてパラメータの最適化、検証データを用いて学習中のモデル検証を行います。`train()` 関数は、検証データに対して最も予測精度の高かったモデルを返します。

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

import copy


def train(
    X,
    y,
    model_class=None,
    model_params=None,
    batch_size=1024,
    epochs=3000,
    criterion=None,
    optimizer_class=None,
    opt_params=None,
    lr_sche_class=None,
    lr_sche_params=None,
):
    X_tensor, y_tensor = (
        torch.from_numpy(X).float(),
        torch.from_numpy(y).float(),
    )
    indices = np.array(range(X.shape[0]))
    indices_train, indices_valid = train_test_split(
        indices, test_size=0.2, random_state=42
    )

    train_set = TensorDataset(X_tensor[indices_train], y_tensor[indices_train])
    valid_set = TensorDataset(X_tensor[indices_valid], y_tensor[indices_valid])
    loaders = {
        "train": DataLoader(train_set, batch_size=batch_size, shuffle=True),
        "valid": DataLoader(valid_set, batch_size=batch_size, shuffle=False),
    }

    model = model_class(**model_params)
    best_model_wts = copy.deepcopy(model.state_dict())
    optimizer = optimizer_class(model.parameters(), **opt_params)
    if lr_sche_class is not None:
        scheduler = lr_sche_class(optimizer, **lr_sche_params)
    best_score = 1e18
    for epoch in range(epochs):
        losses = {"train": 0.0, "valid": 0.0}

        for phase in ["train", "valid"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            for batch_x, batch_y in loaders[phase]:
                optimizer.zero_grad()
                out = model(batch_x).T[0]
                loss = criterion(out, batch_y)
                losses[phase] += loss.item() * batch_x.size(0)

                with torch.set_grad_enabled(phase == "train"):
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

            losses[phase] /= len(loaders[phase].dataset)

        with torch.no_grad():
            model.eval()
            if best_score > losses["valid"]:
                best_model_wts = copy.deepcopy(model.state_dict())
                best_score = losses["valid"]
        if lr_sche_class is not None:
            scheduler.step()

    with torch.no_grad():
        model.load_state_dict(best_model_wts)
        model.eval()
    return model

<a id="2_3"></a>
### 2.3\. 初期教師データの作成

入力値 $\boldsymbol{x}$ に対して目的関数 $f(\boldsymbol{x})$ を評価し、$N_0$個の入出力ペア（初期教師データ）を作成します。ここでの入力値 $\boldsymbol{x}$ の決め方は様々ですが、乱数を用いたり、現象に対する知見に基づき機械学習に適した値を用いたりします。過去に実施した実験やシミュレーションの結果から、教師データを構築しても構いません。

In [ ]:
def gen_training_data(D: int, N0: int, true_func):
    assert N0 < 2**D
    # N0個の入力値を乱数を用いて取得
    X = np.random.randint(0, 2, size=(N0, D))
    # 取得した入力値のうち重複しているものを除外し、除外した分の入力値を乱数を用いて追加
    X = np.unique(X, axis=0)
    while X.shape[0] != N0:
        X = np.vstack((X, np.random.randint(0, 2, size=(N0 - X.shape[0], D))))
        X = np.unique(X, axis=0)
    y = np.zeros(N0)
    # N0個の入力値に対応する出力値を目的関数を評価して取得
    for i in range(N0):
        if i % 10 == 0:
            print(f"Generating {i}-th training data set.")
        y[i] = true_func(X[i])
    return X, y

<a id="2_4"></a>
### 2.4\. FMQA サイクルの実行クラス

`FMQA.cycle()` では、事前に準備した初期教師データを用い、FMQA サイクルを $N-N_0$ 回実施します。`FMQA.step()` は、FMQA を1サイクルのみ行う関数で、`FMQA.cycle()` から $N-N_0$ 回呼び出されます。

In [ ]:
from amplify import (
    Solver,
    BinarySymbolGenerator,
    sum_poly,
    BinaryMatrix,
    BinaryQuadraticModel,
)
import matplotlib.pyplot as plt
import sys


class FMQA:
    def __init__(self, D: int, N: int, N0: int, k: int, true_func, solver) -> None:
        assert N0 < N
        self.D = D
        self.N = N
        self.N0 = N0
        self.k = k
        self.true_func = true_func
        self.solver = solver
        self.y = None

    # 教師データに基づいて N-N0 回のFMQAを教師データを追加しながら繰り返し実施するメンバー関数
    def cycle(self, X, y, log=False) -> np.ndarray:
        print(f"Starting FMQA cycles...")
        pred_x = X[0]
        pred_y = 1e18
        for i in range(self.N - self.N0):
            print(f"FMQA Cycle #{i} ", end="")
            try:
                x_hat = self.step(X, y)
            except RuntimeError:
                sys.exit(f"Unknown error, i = {i}")
            # x_hat として既に全く同じ入力が教師データ内に存在する場合、その周辺の値を x_hat とする。
            is_identical = True
            while is_identical:
                is_identical = False
                for j in range(i + self.N0):
                    if np.all(x_hat == X[j, :]):
                        change_id = np.random.randint(0, self.D, 1)
                        x_hat[change_id.item()] = 1 - x_hat[change_id.item()]
                        if log:
                            print(f"{i=}, Identical x is found, {x_hat=}")
                        is_identical = True
                        break
            # hat{x} で目的関数 f() を評価
            y_hat = self.true_func(x_hat)
            # 最適点近傍における入出力ペア [x_hat, y_hat] を教師データに追加
            X = np.vstack((X, x_hat))
            y = np.append(y, y_hat)
            # 目的関数の評価値が最小値を更新したら、その入出力ペアを [pred_x, pred_y] へコピー
            if pred_y > y_hat:
                pred_y = y_hat
                pred_x = x_hat
                print(f"variable updated, {pred_y=}")
            else:
                print("")
            # 全ての入力を全探索済みの場合は、for文を抜ける
            if len(y) >= 2**self.D:
                print(f"Fully searched at {i=}. Terminating FMQA cycles.")
                break
        self.y = y
        return pred_x

    # 1回のFMQAを実施するメンバー関数
    def step(self, X, y) -> np.ndarray:
        # FM を機械学習
        model = train(
            X,
            y,
            model_class=TorchFM,
            model_params={"d": self.D, "k": self.k},
            batch_size=8,
            epochs=2000,
            criterion=nn.MSELoss(),
            optimizer_class=torch.optim.AdamW,
            opt_params={"lr": 1},
        )
        # 学習済みモデルから、FM パラメータの抽出
        v, w, w0 = list(model.parameters())
        v = v.detach().numpy()
        w = w.detach().numpy()[0]
        w0 = w0.detach().numpy()[0]
        # ここから量子アニーリング・イジングマシンによる求解を実施
        gen = BinarySymbolGenerator()  # BinaryPoly の変数ジェネレータを宣言
        q = gen.array(self.D)  # BinaryPoly から決定変数の作成
        cost = self.__FM_as_QUBO(q, w0, w, v)  # FM パラメータから QUBO として FM を定義
        result = self.solver.solve(cost)  # 目的関数を Amplify のソルバーに受け渡し
        if len(result.solutions) == 0:
            raise RuntimeError("No solution was found.")
        values = result.solutions[0].values
        q_values = q.decode(values)
        return q_values

    # FM パラメータから QUBO として FM を定義する関数。前定義の TorchFM クラスと同様に、g(x) の関数形通りに数式を記述。
    def __FM_as_QUBO(self, x, w0, w, v):
        lin = w0 + (x.T @ w)
        D = w.shape[0]
        out_1 = sum_poly(self.k, lambda i: sum_poly(D, lambda j: x[j] * v[j, i]) ** 2)
        # 次式において、x[j] はバイナリ変数なので、x[j] = x[j]^2 であることに注意。
        out_2 = sum_poly(
            self.k, lambda i: sum_poly(D, lambda j: x[j] * v[j, i] * v[j, i])
        )
        return lin + (out_1 - out_2) / 2

    """上記の __FM_as_QUBO で用いられている sum_poly は、計算速度やメモリの観点から非効率。
    一般的に決定変数の相互作用項が非ゼロである FM の場合、BinaryMatrix を使う次の書き方が効率的。
    ここで、BinaryMatrixでの2次項は、上三角行列で表される非対角項に対応するため、FM式の2次の項に
    対する x(1/2) は不要。また、上の __FM_as_QUBO（sum_poly を使う実装）と関数のシグネチャを
    合わせるために、x を引数に取っているが、BinaryMatrix を使う本実装では本来は不要。
    def __FM_as_QUBO(self, x, w0, w, v):
        out_1_matrix = v @ v.T
        out_2_matrix = np.diag((v * v).sum(axis=1))
        matrix = BinaryMatrix(out_1_matrix - out_2_matrix + np.diag(w))
        # 定数項 w0 を忘れずに BinaryQuadraticModel の2つ目の引数に入れる。
        model = BinaryQuadraticModel(matrix, w0)
        return model
    """

    # 初期教師データ及び各 FMQA サイクル内で実施した i 回の目的関数評価値の履歴をプロットする関数
    def plot_history(self):
        assert self.y is not None
        fig = plt.figure(figsize=(6, 4))
        plt.plot(
            [i for i in range(self.N0)],
            self.y[: self.N0],
            marker="o",
            linestyle="-",
            color="b",
        )  # 初期教師データ生成時の目的関数評価値（ランダム過程）
        plt.plot(
            [i for i in range(self.N0, self.N)],
            self.y[self.N0 :],
            marker="o",
            linestyle="-",
            color="r",
        )  # FMQA サイクル時の目的関数評価値（FMQA サイクル過程）
        plt.xlabel("i-th evaluation of f(x)", fontsize=18)
        plt.ylabel("f(x)", fontsize=18)
        plt.tick_params(labelsize=18)
        return fig

<a id="3"></a>
## 3\. FMQA 実行例

<a id="3_1"></a>
### 3.1\. FMQA による最高臨界温度を実現する材料探索

それでは実装した FMQA 及びモデル関数を用いて、材料探索を行います。今回は、モデルから計算される臨界温度を最大化するので、目的関数として、臨界温度の負値を返すように実装し、この値を最小化するように FMQA を実施します。

以下では、目的関数を評価できる回数 $N$ を100回、そのうち初期データの生成のための評価回数 $N_0=60$ 回としています。従って、以下の例では、$N-N_0=40$ 回、FMQA のサイクル（機械学習、量子アニーリング・イジングマシンによる最適解の求解、目的関数の評価）を実施します。この設定では、FMQA のサイクルが全て終了するまでおよそ5～10分程度の時間がかかりますので、ご注意ください。

In [ ]:
seed_everything()  # 乱数シードの初期化
D = 100  # 決定変数のサイズ（材料選択肢の数）

# 係数テーブルの準備
debye_temperature_table, state_density_table, interaction_table = set_properties(D)

# 目的関数。臨界温度を最大化したいので、臨界温度の負値を返し、この値を最小化するような材料の選び方を FMQA により最適化


def true_func(x):
    if np.sum(x) == 0:
        return 0
    return -supercon_temperature(
        x, debye_temperature_table, state_density_table, interaction_table
    )


N = 100  # 関数を評価できる回数
N0 = 60  # 初期教師データのサンプル数
k = 20  # FMにおけるベクトルの次元（ハイパーパラメータ）

# client：先に作成した Amplify クライアント
solver = Solver(client)
# 初期教師データの生成
X, y = gen_training_data(D, N0, true_func)

# FMQA のインスタンス化
fmqa_solver = FMQA(D, N, N0, k, true_func, solver)
# FMQA サイクルの実行
pred_x = fmqa_solver.cycle(X, y)
# 最適化結果の出力
print("pred x:", pred_x)
print("pred value:", true_func(pred_x))

<a id="3_2"></a>
### 3.2\. FMQA 最適化過程における目的関数値の推移

初期教師データ作成時にランダムに生成した入力値に対して得られた $N_0$​ 個の目標関数値及び $N−N_0$​ サイクルの FMQA 最適化過程における目標関数値の推移を以下にプロットします。

それぞれ、青色及び赤色で示されています。FMQA 最適化サイクルにより得られた、その時点での最適と考えられる入力値（赤線）から、最小の目的関数値が次々と更新される様子が示されています。

一般的に、`FixstarsClient` で採用されているヒューリスティクスというアルゴリズムの原理上、得られる解に再現性はありませんが、サンプルコード内のパラメータ、$N_0=60$、$N-N_0=40$ で求解された材料選択肢の場合、得られる臨界温度は、およそ 50 K となります。

In [ ]:
fig = fmqa_solver.plot_history()

<a id="3_3"></a>
### 3.3\. 本 FMQA サンプルコード実行例

一般的に、`FixstarsClient` で採用されているヒューリスティクスというアルゴリズムの原理上、得られる解に完全な再現性はありませんが、本サンプルコードを実行した際に得られる、典型的な標準出力及び画像出力を以下に紹介します。※得られる値が異なる場合があります。

- 『[3.1\. FMQA による最高臨界温度を実現する材料探索](#3_1)』に記載の FMQA コードを与えられた条件のまま実行すると、次のような標準出力が FMQA サイクルの進捗とともに逐次出力されます。

    ```shell
    Generating 0-th training data set.
    Generating 10-th training data set.
    Generating 20-th training data set.
    Generating 30-th training data set.
    Generating 40-th training data set.
    Generating 50-th training data set.
    Starting FMQA cycles...
    FMQA Cycle #0 variable updated, pred_y=-18.98476017536205
    FMQA Cycle #1 
    FMQA Cycle #2 variable updated, pred_y=-25.897204545387414
    FMQA Cycle #3 variable updated, pred_y=-30.641568733824826
    FMQA Cycle #4 
    FMQA Cycle #5 variable updated, pred_y=-33.23380829087865
    FMQA Cycle #6 
    FMQA Cycle #7 
    FMQA Cycle #8 variable updated, pred_y=-40.97929639761995
    FMQA Cycle #9 
    FMQA Cycle #10 
    FMQA Cycle #11 
    FMQA Cycle #12 
    FMQA Cycle #13 
    FMQA Cycle #14 
    FMQA Cycle #15 
    FMQA Cycle #16 
    FMQA Cycle #17 
    FMQA Cycle #18 variable updated, pred_y=-42.00895340350797
    FMQA Cycle #19 variable updated, pred_y=-47.787495086366945
    FMQA Cycle #20 
    FMQA Cycle #21 variable updated, pred_y=-52.41427395241357
    FMQA Cycle #22 
    FMQA Cycle #23 
    FMQA Cycle #24 
    FMQA Cycle #25 
    FMQA Cycle #26 
    FMQA Cycle #27 
    FMQA Cycle #28 
    FMQA Cycle #29 
    FMQA Cycle #30 
    FMQA Cycle #31 
    FMQA Cycle #32 
    FMQA Cycle #33 
    FMQA Cycle #34 
    FMQA Cycle #35 
    FMQA Cycle #36 
    FMQA Cycle #37 
    FMQA Cycle #38 variable updated, pred_y=-55.425491086604936
    FMQA Cycle #39 
    pred x: [0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1.
    1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.
    1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1.
    1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0.
    1. 0. 1. 1.]
    pred value: -55.425491086604936
    ```

- 『[3.2\. FMQA 最適化過程における目的関数値の推移](#3_2)』に記載の `fmqa_reactor.plot_history()` による出力画像は次のようになります。

  ![history](../figures/fmqa_1_supercon_history.png)

<a id="4"></a>
### 発展：より理解を深めるための練習問題

<a id="4_1"></a>
#### 発展1

ランダムな探索で材料の組み合わせを決定する手法をとる場合、FMQA で得られた臨界温度と同レベルの臨界温度を実現する材料の組み合わせを見つけるには、何回程度の試行が必要でしょうか？ [1.3](#1_3) 節の `n_cycle` を変化させて確認してください。

- **補足**

  1度の機械学習に要する時間を $\tau_{ML}$、最適解の求解に要する時間を $\tau_{QA}$、目的関数の評価に要する時間を $\tau_{eval}$ とすると、一般的に、探索に係る総時間コスト $c_t$ は、次のように記述できます。

  $$
  c_t = N_0 \cdot \tau_{eval} + (N - N_0) \cdot (\tau_{ML} + \tau_{QA} + \tau_{eval} )
  $$

  本サンプルコードでは、臨界温度に対してモデルを用いることで、$\tau_{eval}$ は比較的小さいですが、一般的にブラックボックス最適化を行わなければならない課題では、$\tau_{eval} \gg \tau_{ML}$、$\tau_{eval} \gg \tau_{QA}$ である場合が多いでしょう。その場合、総時間コストは、

  $$
  c_t \sim N \cdot \tau_{eval}
  $$

  となります。つまり、今回の場合、例えば、1回の材料合成＋臨界温度計測に1時間必要で、それを単独24時間体制で実施する場合、$N=100$ の探索では 約 4 日、$N=10000$ の探索では約 1 年必要となります。従って、最適化コストを小さくするには、目的関数の評価回数 $N$ を小さくすることがブラックボックス最適化では優先事項です。
  
  ランダムな探索では、非常に大きな $N$ を用いない限り（つまり $c_t$ が莫大）、一般的には FMQA による最適解に近づいたり、超えたりしないことが示されるでしょう。

<a id="4_2"></a>
#### 発展2

FM 機械学習に関するハイパーパラメータを変更してみましょう。最適化精度や計算時間はどのように変化するでしょうか？

- ヒント：以下に抜粋される `FMQA` クラスの `step()` 関数内のモデル呼び出し内におけるパラメータを変更してみてください。（例：エポック数 `epoch` を 0.1 倍にする）
  ```python
    model = train(
        X,
        y,
        model_class=TorchFM,
        model_params={"d": self.D, "k": self.k},
        batch_size=8,
        epochs=2000,
        criterion=nn.MSELoss(),
        optimizer_class=torch.optim.AdamW,
        opt_params={"lr": 1},
    )
  ```

<a id="4_3"></a>
#### 発展3

FMQA による最適化が活用できそうな業務や身の回りの課題を考えてください。その際、決定変数（入力値）や目的関数は何でしょうか？また目的関数の評価方法はどうなるでしょうか？

> 例：次世代ガスタービン発電施設における燃料ブレンド（水素、天然ガス、合成ガス、アンモニア、水蒸気、再循環ガス等）や熱化学条件の最適化。日々の需要に応じた発電出力を担保しながら、燃料調達コストの低減と汚染物質生成の抑制を実現。目的関数は汚染物質の生成量、燃料コスト及び $($電力需要$-$出力$)^2$ で、その評価は、燃料費の（線形）計算と大規模シミュレーション又は実機による計測。